In [1]:
import numpy as np
import pandas as pd
import ast

credit = pd.read_csv("dataset/credits.csv")
movies = pd.read_csv("dataset/movies.csv")
mv = movies.merge(credit,on="title")
mv.info()
mv = mv[['movie_id','title','overview','genres','keywords','cast','crew']]
mv.dropna(inplace=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [2]:
mv.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [72]:
def toint(obj):
    a = []
    for i in ast.literal_eval(obj):
        a.append(i['name'])
    return a

In [ ]:
mv['genres'] = mv['genres'].apply(toint)
mv.head()

In [ ]:
mv['keywords'] = mv['keywords'].apply(toint)
mv.head()

In [ ]:
mv['cast'][0]

In [76]:
def toint1(obj):
    a = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3 :
            a.append(i['name'])
            counter+=1
        else:
            break
    return a


In [ ]:
mv['cast']=mv['cast'].apply(toint1)
mv.head()

In [78]:
def fetch_dir(obj):
    a = []
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            a.append(i['name'])
            break
    return a

In [ ]:
mv['crew']=mv['crew'].apply(fetch_dir)
mv.head()

In [ ]:
mv['overview'] = mv['overview'].apply(lambda x:x.split())
mv.head()

In [81]:
mv['genres'] = mv['genres'].apply(lambda x:[i.replace(" ","") for i in x])
mv['keywords'] = mv['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
mv['cast'] = mv['cast'].apply(lambda x:[i.replace(" ","") for i in x])
mv['crew'] = mv['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
mv.head()

In [ ]:
mv['tags'] = mv['overview'] + mv['genres'] + mv['keywords'] + mv['cast'] + mv['crew']
mv.head()

In [ ]:
new_mv = mv[['movie_id','title','tags']]
new_mv.head()

In [ ]:
new_mv.info(0)

In [ ]:
new_mv['tags'] = new_mv['tags'].apply(lambda x:" ".join(x))
new_mv.head()

In [87]:
new_mv['tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [ ]:
new_mv['tags'] = new_mv['tags'].apply(lambda x:x.lower())


In [89]:
new_mv.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


In [90]:
import nltk
from nltk.stem.porter import PorterStemmer as ps
ps = ps()

In [91]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)

In [ ]:
new_mv['tags'] = new_mv['tags'].apply(stem)

In [93]:
from sklearn.feature_extraction.text import CountVectorizer as cv
cv = cv(max_features=5000,stop_words='english')

In [94]:
vec = cv.fit_transform(new_mv['tags']).toarray()

In [95]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [96]:
from sklearn.metrics.pairwise import cosine_similarity 


In [97]:
similarity = cosine_similarity(vec)

In [ ]:
similarity[2]

In [173]:
def recommend(mve):
    mve = mve.title()
    if mve == 'Spider Man' or mve == 'Spiderman' :
        mve = 'Spider-Man'
    if  mve == 'Ironman' :
        mve = 'Iron Man'
    mv_index = new_mv[new_mv['title']==mve].index[0]
    distance = similarity[mv_index]
    mv_list = sorted(list(enumerate(distance)),reverse=True,key = lambda x:x[1])[1:11]
    for i in mv_list:
        print(new_mv.iloc[i[0]].title)
    



In [175]:
recommend('Ironman')

Iron Man 3
Iron Man 2
Avengers: Age of Ultron
The Avengers
Captain America: Civil War
Guardians of the Galaxy
X-Men
Thor: The Dark World
Ant-Man
X-Men Origins: Wolverine


In [177]:
import pickle

In [178]:
pickle.dump(new_mv,open('mvs_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

Type the Below code with a movie name to get the recommendation
recommend('Movie_name_of_your_choice)